In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
PATH = '../'

In [3]:
def get_log(fnames, fn, sortby):
    dfs = []
    for fname in fnames:
        dfs.append(fn(fname))
    df = pd.concat(dfs)
    df.sort_values(by=[sortby], inplace=True)
    return df

# Server log

In [4]:
def get_server_log(fname):
    df = pd.read_csv(fname, sep=']', header=None)
    df = df.iloc[:, [0, 8]]
    df.columns = ['end_time', 'duration']
    df['end_time'] = df.end_time.str.replace('[', '')
    df.duration = df.duration.str.replace('duration \[', '')
    df.end_time = pd.to_datetime(df.end_time)
    df['start_time'] = df.end_time - pd.to_timedelta(df.duration)
    df = df[['start_time', 'end_time', 'duration']]
    df = df.rename(columns={'start_time': 'start', 'end_time': 'end'})
    return df

# Slow log

In [5]:
def load_slow_log(fname):
    df = pd.read_csv(fname, sep=']', header=None)
    df = df.iloc[:, [0, 4, 6, 13]]
    df.columns = ['end_time', 'es_index', 'duration', 'is_sort']
    df.end_time = df.end_time.str.replace('[', '')
    df.es_index = df.es_index.str.replace('[', '')
    df.duration = df.duration.str.replace('took\[', '')
    df.is_sort = df.is_sort.apply(lambda val: True if 'sort' in val else False)
    df.end_time = pd.to_datetime(df.end_time)
    df['start_time'] = df.end_time - pd.to_timedelta(df.duration)
    df = df[['start_time', 'end_time', 'duration', 'es_index', 'is_sort']]
    df = df.rename(columns={'start_time': 'start', 'end_time': 'end'})
    return df

# Validation

In [6]:
def are_subsequent(first, second=None):
    if second is None:
        res = [first.iloc[i] <= first.iloc[i+1] for i in range(len(first)-1)]
        return all(res), res
    intervals = []
    for s, e in zip(first, second):
        intervals.append(s)
        intervals.append(e)
    res = [intervals[i] <= intervals[i+1] for i in range(len(intervals)-1)]
    return all(res), res

# Compare

In [7]:
def print_stats(server_log, slow_log):
    # Before first gc
    print('{}\t{}\t\t\t\t\t{}'.format(0, 'n/a', len(slow_log[slow_log.start < server_log.iloc[0].start])))
    # Between first-second, second-third, etc
    for i in range(len(server_log)-1):
        prev_gc_start = server_log.iloc[i].start
        gc_start = server_log.iloc[i+1].start
        # sort queries before gc and after previous gc
        df = slow_log[(slow_log.start <= gc_start) & (slow_log.start >= prev_gc_start)]
        print('{}\t{}\t{}\t{}'.format(i+1, gc_start, server_log.iloc[i+1].duration, len(df[['start', 'end']])))

# Hypothesis
There is initial burst of sort queries, which triggers gc, the unprocessed queries are queued and after gc they cause anohter gc and another, resulting in long jvm pause, after/during which a node can not communicate with master and disconnects or get disconnected from the cluster.

# Tests

In [8]:
def extract_gc_lines(path):
    server_logs = ['{}/{}'.format(path, name) for name in os.listdir(path) if 'insight-neo.log' in name]
    for i, server_log in enumerate(server_logs):
        wrote = False
        gc_log = '{}/{}.gc'.format(path, i)
        with open(server_log) as fr:
            with open(gc_log, 'w+') as fw:
                for line in fr:
                    if '[gc]' in line and 'duration' in line:
                        fw.write(line)
                        wrote = True
        if not wrote:
            os.remove(gc_log)

In [9]:
def run(path, should_raise=True):
    extract_gc_lines(path)
    server_logs = ['{}/{}'.format(path, name) for name in os.listdir(path) if name.endswith('gc')]
    server_log = get_log(server_logs, get_server_log, 'start')
    slow_logs = ['{}/{}'.format(path, name) for name in os.listdir(path) if 'slowlog' in name]
    slow_log = get_log(slow_logs, load_slow_log, 'start')
    assert1 = [are_subsequent(e) for e in [slow_log.start, slow_log.end, server_log.start, server_log.end]]
    expected1 = [True, False, True, True]
    compare(assert1, expected1, should_raise)
    assert2 = [are_subsequent(*e) for e in [(slow_log.start, slow_log.end), (server_log.start, server_log.end)]]
    expected2 = [False, True]
    compare(assert2, expected2, should_raise)
    
    server_log = server_log[(server_log.start >= slow_log.iloc[0].start)]
    
    print_stats(server_log, slow_log)

In [10]:
def compare(actual, expected, should_raise=True):
    try:
        for i, a in enumerate(actual):
            assert expected[i] == a[0]
    except AssertionError:
        print('At index {} it was expected {} but it was {}. Complete list: {}'.format(
            i, expected[i], a[0], a[1]))
        if should_raise:
            raise

In [11]:
run(PATH+'54-gc-logs')

0	n/a					173
1	2018-08-01 13:26:35.127000	 1.2m	28
2	2018-08-01 13:27:47.427000	 57.5s	0
3	2018-08-01 13:28:51.538000	 1.2m	4
4	2018-08-01 13:30:03.988000	 58s	0
5	2018-08-01 13:31:03.191000	 1.3m	1
6	2018-08-01 13:32:21.520000	 58.4s	1
7	2018-08-01 13:33:20.407000	 1.3m	0
8	2018-08-01 13:34:38.717000	 58.1s	0
9	2018-08-01 13:35:41.860000	 1.2m	0
10	2018-08-01 13:36:54.034000	 56.4s	0
11	2018-08-01 13:37:55.001000	 1.2m	2
12	2018-08-01 13:39:12.317000	 11.6m	1
13	2018-08-01 13:50:51.214000	 1.4m	0
14	2018-08-01 13:52:15.588000	 58.9s	0


In [12]:
run(PATH+'05-gc-logs')

0	n/a					101
1	2018-08-01 13:26:02.825000	 1.1m	46
2	2018-08-01 13:27:09.399000	 55.9s	3
3	2018-08-01 13:28:10.527000	 1.2m	5
4	2018-08-01 13:29:22.877000	 56.6s	0
5	2018-08-01 13:30:23.443000	 1.2m	0
6	2018-08-01 13:31:35.737000	 56.6s	0
7	2018-08-01 13:32:36.153000	 1.2m	0
8	2018-08-01 13:33:48.522000	 56.7s	0
9	2018-08-01 13:34:49.213000	 1.2m	0
10	2018-08-01 13:36:01.364000	 56.6s	0
11	2018-08-01 13:37:01.816000	 1.2m	0
12	2018-08-01 13:38:14.029000	 57s	0
13	2018-08-01 13:39:14.779000	 5.3m	0


In [13]:
run(PATH+'55-gc-logs', should_raise=False)

At index 1 it was expected True but it was False. Complete list: [True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
0	n/a					90
1	2018-08-01 13:26:29.991000	 789ms	7
2	2018-08-01 13:26:38.749000	 1m	2
3	2018-08-01 13:27:38.031000	 718ms	0
4	2018-08-01 13:28:30.497000	 749ms	58
5	2018-08-01 13:28:32.632000	 1.2m	0
6	2018-08-01 13:29:45.164000	 57s	0
7	2018-08-01 13:30:47.599000	 1.2m	0
8	2018-08-01 13:32:00.176000	 57.8s	0
9	2018-08-01 13:33:01.158000	 1.4m	0
10	2018-08-01 13:34:25.427000	 58s	0
11	2018-08-01 13:35:23.779000	 1.3m	0
12	2018-08-01 13:36:41.916000	 58.3s	0
13	2018-08-02 13:07:05.011000	 15s	61
14	2018-08-03 06:33:10.138000	 20.5s	0


In [14]:
run(PATH+'15-gc-logs')

0	n/a					153
1	2018-08-03 07:37:15.232000	 19.5s	146


In [15]:
run(PATH+'04-gc-logs')

0	n/a					296


In [16]:
run(PATH+'57-gc-logs')

0	n/a					110
1	2018-08-03 12:43:13.764000	 1.1m	73
2	2018-08-03 12:44:26.362000	 1.2m	1
3	2018-08-03 12:45:38.944000	 57.7s	0


In [17]:
run(PATH+'09-gc-logs')

0	n/a					129
1	2018-08-03 12:45:11.187000	 58.5s	47
2	2018-08-03 12:46:15.071000	 1.2m	0
3	2018-08-03 12:47:27.526000	 56.9s	0
4	2018-08-03 12:48:27.845000	 1.3m	0
5	2018-08-03 12:49:46.257000	 58s	0
